### Working with a jobs

The job class obtained after running a QuantumCircuit has interesting properties that can be used.

In [30]:
# !python -m pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple c12simulator-clients -q

from c12simulator_clients.user_configs import UserConfigs
from c12simulator_clients.qiskit_back.c12sim_provider import C12SimProvider
from qiskit import QuantumCircuit

TOKEN = "db0ccae9b0dfccba90a534ad40802d40aa57d395bdac4e3a0bfcaaa7db0a3c2f"
configs = UserConfigs.parse_obj({"token" : TOKEN})

c12_simulator_provider = C12SimProvider(configs)
c12_simulator_backend = c12_simulator_provider.get_backend('c12sim')


circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)

circuit.draw()

┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘

Two main methods available for each Job instance are job_id() and status().

1. `job_id()` method returns the unique identifier of a job as a UUID4 string (
 a universally unique identifier (UUID), version 4,  is a 36-character alphanumeric random generated string ). This id can be used for later assessment of the job properties.

2. `status()` method is used to get the status of a job execution as an instance of JobStatus class. Available statuses are:
    - `QUEUED` = Job is queued. It waits for execution
    - `RUNNING` = Job is currently running.
    - `CANCELLED` = Job has been canceled.
    - `DONE` = Job has been successfully done.
    - `ERROR` = There has been an error during the execution of a job.

The status of a running job can be changed depending on the current state of job execution.


In [32]:
c12_job = c12_simulator_backend.run(circuit)
print(f"Job id: {c12_job.job_id()}") # Get a job UUID
print(f"Status: {c12_job.status()}")  # Get a current job status

Job id: 47fdc659-5554-431d-b7f6-aadb9cfd94d9
Status: JobStatus.RUNNING


The result of a job execution can be obtained with the `result()` method. This method can have one optional argument (timeout). The `timeout` argument specifies how long the method will wait for the execution of the task. If the time limit is exceeded, the `C12SimJobError` exception is raised. If we do not specify this argument, its default value is None, meaning the method will block until the simulation is finished.

Another way to get the results is to check the job status periodically until it is `DONE`  and then call the `result()` function.

In [33]:
from qiskit.providers import JobStatus
import time
job_final_states = [JobStatus.DONE, JobStatus.ERROR, JobStatus.CANCELLED]
c12_job_id = c12_job.job_id()
while True:
    job_status = c12_job.status()
    print(f'{c12_job_id}: {job_status}')
    if job_status in job_final_states:
        break
    time.sleep(5) # Wait 5 s

if c12_job.status() == JobStatus.DONE:
    c12_result = c12_job.result()
    print(c12_result.get_counts())

47fdc659-5554-431d-b7f6-aadb9cfd94d9: JobStatus.RUNNING
47fdc659-5554-431d-b7f6-aadb9cfd94d9: JobStatus.RUNNING
47fdc659-5554-431d-b7f6-aadb9cfd94d9: JobStatus.RUNNING
47fdc659-5554-431d-b7f6-aadb9cfd94d9: JobStatus.RUNNING
47fdc659-5554-431d-b7f6-aadb9cfd94d9: JobStatus.RUNNING
47fdc659-5554-431d-b7f6-aadb9cfd94d9: JobStatus.RUNNING
47fdc659-5554-431d-b7f6-aadb9cfd94d9: JobStatus.RUNNING
47fdc659-5554-431d-b7f6-aadb9cfd94d9: JobStatus.RUNNING
47fdc659-5554-431d-b7f6-aadb9cfd94d9: JobStatus.DONE
{'01': 1, '11': 529, '00': 494}


In [34]:
# Running a job that will block until finished
c12_job = c12_simulator_backend.run(circuit)
c12_result = c12_job.result()
if c12_job.status() == JobStatus.DONE:
    print(c12_result.get_counts())

{'01': 1, '11': 484, '00': 539}


In [35]:
# Getting a job result with a timeout argument specified
from c12simulator_clients.qiskit_back.exceptions import C12SimJobError
c12_job = c12_simulator_backend.run(circuit)

try:
    c12_result = c12_job.result(timeout=15) # it will raise a Timeout Error as it won't be finished in 15s
except C12SimJobError:
    print("Timeout!")
    print(f" Last status: {c12_job.status()}")

Timeout!
 Last status: JobStatus.DONE


### Getting previous jobs

Next, extremely useful functionality is the possibility to obtain all jobs run on the system by a specific user. Using this functionality, the user can get the results of old jobs.

This can be achieved by calling the `jobs()` function on the backend instance. This function accepts two arguments: `limit` and `offset`. We can specify how many jobs we want to get with a `limit` argument. Likewise, we can set the `offset` from the first job with an offset argument. With these arguments, pagination is easily achieved.

In [37]:
number_of_records = 10
offset = 0
counter = 1
while True:
    jobs = c12_simulator_backend.jobs(number_of_records, offset)

    for job in jobs:
        print(f"{counter}: {job.job_id()} Status: {job.status()} Counts: {job.result().get_counts()}")
        counter += 1

    if len(jobs) < number_of_records:
        break

    offset = offset + number_of_records


1: c9052ef0-4a4d-4b4c-b9e9-4ead90d6ca4b Status: JobStatus.DONE Counts: {'01': 271, '10': 257, '00': 243, '11': 253}
2: a948abbd-057b-4212-b254-a8871f92f9e7 Status: JobStatus.DONE Counts: {'10': 265, '00': 281, '11': 244, '01': 234}
3: 18973d16-3f58-4bce-a74d-d851909edeba Status: JobStatus.DONE Counts: {'01': 261, '11': 249, '00': 263, '10': 251}
4: 015aa6c2-f42f-4f73-9954-bc213c71b26b Status: JobStatus.DONE Counts: {'01': 3, '11': 494, '00': 527}
5: 0a3bf6e5-ba9c-4445-9972-d0e5ae8c937b Status: JobStatus.DONE Counts: {'10': 2, '00': 1022}
6: 955f1cbe-570e-41e1-a50a-8c01f02b9e75 Status: JobStatus.DONE Counts: {'10': 1, '00': 1023}
7: 9f07fae4-6719-4ee2-a0a7-23d3029f7357 Status: JobStatus.DONE Counts: {'01': 1, '10': 4, '00': 1019}
8: 29650169-02f5-4259-b1cc-1ecb0190bcc2 Status: JobStatus.DONE Counts: {'11': 1, '10': 2, '00': 1021}
9: 2d4071b4-cd87-4f6d-be7e-8d6e20bfde56 Status: JobStatus.DONE Counts: {'11': 492, '00': 532}
10: a65dd5e7-9cb0-4e6a-817b-2b07aa20ca4b Status: JobStatus.DONE C

### Running multiple jobs at once

The `run()` method of the backend class can also accept the list of QuantumCircuit. This can be an essential feature as it allows a user to run multiple circuits simultaneously and get an array of job instances as a result.

In [39]:
# Array where the circuits will be stored
circuits_to_run = []

# Creating the circuits
circuit_1 = QuantumCircuit(2)
circuit_1.h(0)
circuit_1.cx(0, 1)

circuit_2 = QuantumCircuit(3)
circuit_2.h(0)
circuit_2.cx(0, 1)
circuit_2.x(2)
circuit_2.h(2)

circuits_to_run.append(circuit_1)
circuits_to_run.append(circuit_2)

# Running the jobs
c12_jobs = c12_simulator_backend.run(circuits_to_run)

# Printing the UUID of the jobs
for job in c12_jobs:
    print(f"Job id: {job.job_id()} - >  {job.status()}")

# Waiting for the results
for job in c12_jobs:
    result = job.result()
    print(result.get_statevector())

Job id: 282fda89-b843-4be7-b0e3-2230ec86c474 - >  JobStatus.RUNNING
Job id: d003f9d3-2362-444a-84b4-f18cdea64df9 - >  JobStatus.QUEUED
[ 1.43483218e-06-7.07170237e-01j -7.23495740e-05+7.45804289e-05j
 -1.50334738e-05+7.94950543e-05j -4.24279316e-06-7.07043308e-01j]
[-1.01462838e-06+5.00068929e-01j  5.11613358e-05-5.27388644e-05j
  1.06307827e-05-5.62141965e-05j  3.00025216e-06+4.99979172e-01j
  1.01453074e-06-5.00020810e-01j -5.11564128e-05+5.27337896e-05j
 -1.06297597e-05+5.62087873e-05j -2.99996346e-06-4.99931061e-01j]
